# Introduction
[Chainer](http://chainer.org/) とはニューラルネットの実装を簡単にしたフレームワークです。

* 今回は機械翻訳にニューラルネットを適用してみました。

![](./pictures/Chainer.jpg)

* 今回は機械翻訳を行っていただきます。


機械翻訳は機械が言語を別の言語に翻訳するものです。

機械翻訳にはいくつか種類があるのでここでも紹介しておきます。

* PBMT(Phrase Base Machine Translation)モデル
 * [moses](http://www.statmt.org/moses/)というオープンソースで使用できるメジャーな機械翻訳のモデルですが、難しすぎて理解できない人を続出させる機械翻訳の鬼門です
* ニューラル機械翻訳
 * 翻訳元単語の辞書ベクトルを潜在空間ベクトルに落とし込み、ニューラルネットで翻訳先言語を学習させる手法

以下では、このChainerを利用しデータを準備するところから実際にNN翻訳モデルを構築し学習・評価を行うまでの手順を解説します。

<A HREF=#1.各種ライブラリ導入 >1.各種ライブラリ導入</A><br>
<A HREF=#2.機械翻訳のクラス >2.機械翻訳のクラス</A><br>
<A HREF=#3.翻訳処理を行うforwardに必要なパラメータ設定 >3.翻訳処理を行うforwardに必要なパラメータ設定</A><br>
<A HREF=#4.翻訳処理を行うEncoder処理部分 >4.翻訳処理を行うEncoder処理部分</A><br>
<A HREF=#5.翻訳処理を行うDecoder処理部分 >5.翻訳処理を行うDecoder処理部分</A><br>
<A HREF=#6.翻訳処理を行うforward処理部分 >6.翻訳処理を行うforward処理部分</A><br>
<A HREF=#7.各値を設定 >7.各値を設定</A><br>
<A HREF=#8.実行 >8.実行</A><br>
<A HREF=#9.学習したモデルを使用したテスト >9.学習したモデルを使用したテスト</A><br>
<A HREF=#10.学習したモデルを評価 (Advanced) >10.学習したモデルを評価 (Advanced)</A><br>

##  <A NAME=1.各種ライブラリ導入 /> 1.各種ライブラリ導入

Chainerの言語処理では多数のライブラリを導入します。
Ctrl → m → lをコードの部分で入力すると行番号が出ます。ハンズオンの都合上、行番号があった方が良いので対応よろしくお願いします。

In [ ]:
#表示用に使用しています。
from util.functions import trace
import numpy as np

from chainer import functions, optimizers

#cpu計算とgpu計算で使い分けるラッパー
from util.chainer_cpu_wrapper import wrapper

from EncoderDecoderModel import EncoderDecoderModel
import subprocess

`導入するライブラリの代表例は下記です。

* `numpy`: 行列計算などの複雑な計算を行なうライブラリ
* `chainer`: Chainerの導入
* `util`:今回の処理で必要なライブラリが入っています。


##  <A NAME=2.機械翻訳のクラス /> 2.機械翻訳のクラス

下記を設定しています。
* ニューラルネットを用いて機械翻訳用のモデルを構成しています。

全体構成

![](./pictures/NN_machine_translation.png)




##  <A NAME=3.翻訳処理を行うforwardに必要なパラメータ設定 /> 3.翻訳処理を行うforwardに必要なパラメータ設定

下記のコードで必要なパラメータを設定するクラスを定義しています。

In [ ]:
class EncoderDecoderModelParameter():
    
    def __init__(self, is_training, src_batch, encoderDecoderModel, trg_batch = None, generation_limit = None):
        self.model = encoderDecoderModel.model
        self.tanh = functions.tanh
        self.lstm = functions.lstm
        self.batch_size = len(src_batch)
        self.src_len = len(src_batch[0])
        #翻訳元言語を単語からインデックスにしている（ニューラルネットの空間で扱うため）
        self.src_stoi = encoderDecoderModel.src_vocab.stoi
        #翻訳先言語を単語からインデックスにしている（ニューラルネットの空間で扱うため）
        self.trg_stoi = encoderDecoderModel.trg_vocab.stoi
        #翻訳先言語をインデックスから単語にしている(翻訳結果として保持するため、翻訳先言語だけ用意している)
        self.trg_itos = encoderDecoderModel.trg_vocab.itos
        #lstmのために状態を初期化
        self.state_c = wrapper.zeros((self.batch_size, encoderDecoderModel.n_hidden))
        self.trg_batch = trg_batch
        self.generation_limit = generation_limit

##  <A NAME=4.翻訳処理を行うEncoder処理部分 /> 4.翻訳処理を行うEncoder処理部分

下記の論文を参考にしてforward処理を記述しています。

http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf

* Encoder部分
もっとも特徴的な部分は翻訳元言語を逆順にしていることです。そうすることで精度が向上していると述べており、今回の翻訳のNNモデルもそれを再現しています。

この論文でははっきりした要因はわかっていないが、おそらく翻訳先の言語と翻訳元言語の距離が逆順にすることで最初の単語の距離が近くなり、翻訳のタイムラグが少なくなったことが起因していると考えられています。

![](./pictures/encoder.png)



In [ ]:
class EncoderDecoderModelEncoding():
    
    def encoding(self, src_batch, parameter, trg_batch = None, generation_limit = None):

#--------Hands on------------------------------------------------------------------#
    # encoding
        #翻訳元言語の末尾</s>を潜在空間に射像し、隠れ層に入力、lstmで出力までをバッチサイズ分行う
        #予め末尾の設定をしていないと終了単語が分からないため
        #1:翻訳元言語の入力x:図のx部分に相当
        state_x = wrapper.make_var([parameter.src_stoi('</s>') for _ in range(parameter.batch_size)], dtype=np.int32)
        #2:翻訳元言語の入力xを潜在空間に射像する。（次元数を圧縮するため）:図のi部分に相当
        state_i = parameter.tanh(parameter.model.weight_xi(state_x))
        #3:潜在空間iの入力をlstmに入力し、次の単語予測に使用する:図のp部分に相当
        parameter.status_c, state_p = parameter.lstm(parameter.state_c, parameter.model.weight_ip(state_i))
        
        #翻訳元言語を逆順に上記と同様の処理を行う   
        for l in reversed(range(parameter.src_len)):
            #翻訳元言語を語彙空間に写像
            state_x = wrapper.make_var([parameter.src_stoi(src_batch[k][l]) for k in range(parameter.batch_size)], 
                                       dtype=np.int32)
            #語彙空間を潜在空間（次元数が減る）に射像
            state_i = parameter.tanh(parameter.model.weight_xi(state_x))
            #状態と出力結果をlstmにより出力。lstmの入力には前の状態と語彙空間の重み付き出力と前回の重み付き出力を入力としている
            parameter.state_c, state_p = parameter.lstm(parameter.status_c, parameter.model.weight_ip(state_i) 
                                                        + parameter.model.weight_pp(state_p))

        #次のミニバッチ処理のために最終結果をlstmで出力。翻訳の仮説用のリストを保持
        parameter.state_c, state_q = parameter.lstm(parameter.state_c, parameter.model.weight_pq(state_p))
        hyp_batch = [[] for _ in range(parameter.batch_size)]
        return state_q, hyp_batch
#--------Hands on------------------------------------------------------------------#

##  <A NAME=5.翻訳処理を行うDecoder処理部分 /> 5.翻訳処理を行うDecoder処理部分

* Decoder部分

学習部分と予測部分を実装しています。学習部分ではターゲット先の単語の取得と損失の計算をしています。
またlstmで次回の学習に使用する部分では学習では正解の翻訳、予測では予測した翻訳を使用しています。

![](./pictures/decorder.png)

In [ ]:
class EncoderDecoderModelDecoding():
    
    def decoding(self, is_training, src_batch, parameter, state_q, hyp_batch, trg_batch = None, generation_limit = None):

#--------Hands on------------------------------------------------------------------#
    # decoding
        """
　　     学習
        """
        if is_training:
            #損失の初期化及び答えとなる翻訳先言語の長さを取得。（翻訳元言語と翻訳先言語で長さが異なるため）
            #損失が最小となるように学習するため必要
            accum_loss = wrapper.zeros(())
            trg_len = len(parameter.trg_batch[0])

            #ニューラルネットの処理は基本的にEncodingと同一であるが、損失計算と翻訳仮説候補の確保の処理が加わっている
            for l in range(trg_len):
                #1:翻訳元言語に対するニューラルの出力qを受け取り、潜在空間jに射像
                state_j = parameter.tanh(parameter.model.weight_qj(state_q))
                #2:潜在空間jから翻訳先言語yの空間に射像
                result_y = parameter.model.weight_jy(state_j)
                #3:答えとなる翻訳結果を取得
                state_target = wrapper.make_var([parameter.trg_stoi(parameter.trg_batch[k][l]) 
                                                 for k in range(parameter.batch_size)], dtype=np.int32)
                #答えと翻訳結果により損失を計算
                accum_loss += functions.softmax_cross_entropy(result_y, state_target)
                #複数翻訳候補が出力されるため、出力にはもっとも大きな値を選択
                output = wrapper.get_data(result_y).argmax(1)

                #翻訳仮説確保(インデックスから翻訳単語に直す処理も行っている）
                for k in range(parameter.batch_size):
                    hyp_batch[k].append(parameter.trg_itos(output[k]))

                #状態と出力結果をlstmにより出力。lstmの入力には前の状態と語彙空間の重み付き出力と前回の重み付き出力を入力としている
                parameter.status_c, state_q = parameter.lstm(parameter.status_c, parameter.model.weight_yq(state_target) 
                                                             + parameter.model.weight_qq(state_q))
            return hyp_batch, accum_loss
        else:
            """
            予測部分
            """
            #末尾に</s>が予測できないと無限に翻訳してしまうため、予測では予測する翻訳言語の長さに制約をしている
            while len(hyp_batch[0]) < parameter.generation_limit:
                state_j = parameter.tanh(parameter.model.weight_qj(state_q))
                result_y = parameter.model.weight_jy(state_j)
                #複数翻訳候補が出力されるため、出力にはもっとも大きな値を選択
                output = wrapper.get_data(result_y).argmax(1)

                #翻訳仮説確保(インデックスから翻訳単語に直す処理も行っている）
                for k in range(parameter.batch_size):
                    hyp_batch[k].append(parameter.trg_itos(output[k]))

                #ミニバッチサイズ分の翻訳仮説の末尾が</s>になったときにDecoding処理が終わるようになっている。
                if all(hyp_batch[k][-1] == '</s>' for k in range(parameter.batch_size)): break
                
                #翻訳仮説をニューラルネットで扱える空間に射像している
                state_y = wrapper.make_var(output, dtype=np.int32)
                #次のlstmの処理のために出力結果と状態を渡している
                parameter.status_c, state_q = parameter.lstm(parameter.state_c, parameter.model.weight_yq(state_y) 
                                                             + parameter.model.weight_qq(state_q))

            return hyp_batch
        
#--------Hands on------------------------------------------------------------------#

##  <A NAME=6.翻訳処理を行うforward処理部分 /> 6.翻訳処理を行うforward処理部分

上記の処理を実行するためのメソッドです。

In [ ]:
class EncoderDecoderModelForward(EncoderDecoderModel):
    
    def forward(self, is_training, src_batch, trg_batch = None, generation_limit = None):
    #パラメータ設定
        parameter = EncoderDecoderModelParameter(is_training, src_batch, self, trg_batch, generation_limit)
        
    # encoding
        encoder = EncoderDecoderModelEncoding()
        s_q, hyp_batch = encoder.encoding(src_batch, parameter)
    # decoding
        decoder = EncoderDecoderModelDecoding()
        if is_training:
            return decoder.decoding(is_training, src_batch, parameter, s_q, hyp_batch, trg_batch, generation_limit)
        else:
            return decoder.decoding(is_training, src_batch, parameter, s_q, hyp_batch, trg_batch, generation_limit)

##  <A NAME=7.各値を設定 /> 7.各値を設定

各値を設定

* 翻訳元言語の設定(学習データ)
* 翻訳先言語の設定（学習データ）
* 翻訳元言語の設定(テストデータ)
* 翻訳先言語の設定（テストデータ）
* 語彙の設定
* 潜在空間の設定
* 隠れ層の設定
* 学習回数の設定
* ミニバッチサイズの設定
* 最大予測言語数の設定
ベストな調整方法は経験則か力技です。グリッドサーチ、ランダムサーチ、データから推定など。

In [ ]:
parameter_dict = {}
train_path = "oda201512handson/train/"
test_path = "oda201512handson/test/"
parameter_dict["source"] = train_path + "train1000.ja"
parameter_dict["target"] = train_path + "train1000.en"
parameter_dict["test_source"] = test_path + "test1000.ja"
parameter_dict["test_target"] = test_path + "test1000_hyp.en"
parameter_dict["reference_target"] = test_path + "test1000.en"
#--------Hands on  2----------------------------------------------------------------

"""
下記の値が大きいほど扱える語彙の数が増えて表現力が上がるが計算量が爆発的に増えるので大きくしない方が良いです。
"""
parameter_dict["vocab"] = 550

"""
この数が多くなればなるほどモデルが複雑になります。この数を多くすると必然的に学習回数を多くしないと学習は
収束しません。
語彙数よりユニット数の数が多いと潜在空間への写像が出来ていないことになり結果的に意味がない処理になります。
"""
parameter_dict["embed"] = 500

"""
この数も多くなればなるほどモデルが複雑になります。この数を多くすると必然的に学習回数を多くしないと学習は
収束しません。
"""
parameter_dict["hidden"] = 20

"""
学習回数。基本的に大きい方が良いが大きすぎると収束しないです。
"""
parameter_dict["epoch"] = 20

"""
ミニバッチ学習で扱うサイズです。この点は経験的に調整する場合が多いが、基本的に大きくすると学習精度が向上する
代わりに学習スピードが落ち、小さくすると学習精度が低下する代わりに学習スピードが早くなります。
"""
parameter_dict["minibatch"] = 64

"""
予測の際に必要な単語数の設定。長いほど多くの単語の翻訳が確認できるが、一般的にニューラル翻訳は長い翻訳には
向いていないので小さい数値がオススメです。
"""
parameter_dict["generation_limit"] = 256

#--------Hands on  2----------------------------------------------------------------#

##  <A NAME=8.実行 /> 8.実行

In [ ]:
trace('initializing ...')
wrapper.init()

encoderDecoderModel = EncoderDecoderModelForward(parameter_dict)
encoderDecoderModel.train_model()

##  <A NAME=9.学習したモデルを使用したテスト /> 9.学習したモデルを使用したテスト 

学習したモデルを使用してテスト

* 学習したモデルを利用してテストデータ（日本語）を英語に翻訳しモデルに保存。

In [ ]:
model_name = "ChainerMachineTranslation.021"
trace('initializing ...')
wrapper.init()

encoderDecoderModel = EncoderDecoderModelForward(parameter_dict)
encoderDecoderModel.test_model(model_name)

##  <A NAME=10.学習したモデルを評価 (Advanced) /> 10.学習したモデルを評価 (Advanced)

学習したモデルの評価するため、BLEUを算出

* BlEUとは翻訳の客観的評価に使用される指標で、答えとなる文章との一致率を評価する方法を用いています。
詳しく知りたい方は下記をご覧ください。
http://www2.nict.go.jp/univ-com/multi_trans/member/mutiyama/corpmt/4.pdf

In [ ]:
cmd_corpus = "mteval-corpus  -e BLEU RIBES -r " +parameter_dict["reference_target"] + " -h " + parameter_dict["test_target"]
cmd_sentence = "mteval-sentence  -e BLEU RIBES -r " +parameter_dict["reference_target"] + " -h " + parameter_dict["test_target"]
mteval_corpus = subprocess.Popen(cmd_corpus, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout_data = mteval_corpus.stdout.read()
print(stdout_data.decode('utf-8'))